In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import evaluate
from transformers import AutoTokenizer

/home/leroy/.pyenv/versions/3.12.3/envs/data-analysis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset = pd.read_csv('../datasets/dataset.csv')

print(dataset.__len__())

723996


In [7]:
X = dataset['tweet']
y = dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

accuracy = evaluate.load('accuracy')